In [181]:
from google.cloud import bigquery
from datetime import datetime

client = bigquery.Client()

project_id = 'snsdisasteralert-372409'
client = bigquery.Client(project=project_id)

# Insert values in a table
from google.cloud import bigquery
# client = bigquery.Client()

dataset_id = "sns"
# For this sample, the table must already exist and have a defined schema
table_id = "tweets"
table_ref = client.dataset(dataset_id).table(table_id)
table = client.get_table(table_ref)


## Reference SQL syntax from the original job
Use the ```jobs.query```
[method](https://cloud.google.com/bigquery/docs/reference/rest/v2/jobs/query) to
return the SQL syntax from the job. This can be copied from the output cell
below to edit the query now or in the future. Alternatively, you can use
[this link](https://console.cloud.google.com/bigquery?j=snsdisasteralert-372409:EU:bquxjob_70670b5_185c3f53c67)
back to BigQuery to edit the query within the BigQuery user interface.

In [182]:
# Running this code will display the query used to generate your previous job

job = client.get_job('bquxjob_f1a686f_185d3ff12d7') # Job ID inserted based on the query results selected to explore
print(job.query)

SELECT  id,author_id,text,max(pb.like_count) as like_count,max(pb.retweet_count) as retweet_count,max(pb.quote_count) as quote_count ,max(followers_count) as followers_count,max(following_count) as following_count,

 DATE_TRUNC( CAST(created_at AS TIMESTAMP), DAY) as tweet_date,
      lower(tag) as tag,sentiment,(max(pb.like_count) + max(pb.quote_count) + max(pb.retweet_count)) * (
CASE sentiment
      WHEN 'negative' THEN -1
      WHEN 'neutral' THEN 0
      WHEN 'positive'  THEN 1
      ELSE 1
   END  ) as virality,ARRAY_TO_STRING(context,", ") as context
FROM `snsdisasteralert-372409.sns.tweets`, UNNEST (public_metrics) as pb
WHERE
(DATE_TRUNC( CAST(created_at AS TIMESTAMP), DAY)  BETWEEN '2022-01-01 00:00:00 UTC' and '2023-01-01 00:00:00 UTC') and
sentiment is not null and 
(
LOWER(tag) = 'tesla' or LOWER(tag) = 'mcdonalds' or Lower(tag) ='twitter' or LOWER(tag) = 'amazon' or LOWER(tag) = 'apple' or LOWER(tag) = 'coca cola' or LOWER(tag) ='walmart' or LOWER(tag) = 'disney' or LOWER

# Result set loaded from BigQuery job as a DataFrame
Query results are referenced from the Job ID ran from BigQuery and the query
does not need to be re-run to explore results. The ```to_dataframe```
[method](https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.job.QueryJob.html#google.cloud.bigquery.job.QueryJob.to_dataframe)
downloads the results to a Pandas DataFrame by using the BigQuery Storage API.

To edit query syntax, you can do so from the BigQuery SQL editor or in the
```Optional:``` sections below.

In [183]:
# Running this code will read results from your previous job

job = client.get_job('bquxjob_f1a686f_185d3ff12d7') # Job ID inserted based on the query results selected to explore
results = job.to_dataframe()
results

,id,author_id,text,like_count,retweet_count,quote_count,followers_count,following_count,tweet_date,tag,sentiment,virality,context
0,1569353177249579010,19003754,I know people who got 20M TikTok views and mad...,3,434,46,74198,15098,2022-09-12 00:00:00+00:00,google,positive,483,"Brand, Unified Twitter Taxonomy, Technology, E..."
1,1523726955211853824,129009730,Africa's Best Ecosystems for Tech Developers\n...,9,717,174,872815,1,2022-05-09 00:00:00+00:00,google,positive,900,"Place, Events [Entity Service], Entities [Enti..."
2,1552772290445807616,1366951308364808193,I'm sure y'all have seen the warnings but Pro*...,4,365,124,4921,433,2022-07-28 00:00:00+00:00,google,positive,493,"Place, Entities [Entity Service], Brand, Produ..."
3,1546543211375648768,1255226720938078208,I had the pleasure of Photographing Dr. Cindy ...,142,5507,1382,10909,675,2022-07-11 00:00:00+00:00,google,positive,7031,"Place, Entities [Entity Service], Brand, Produ..."
4,1566496868716216323,1519675042237489157,To all The Hotep’s out there trying to blame t...,4,519,150,5130,1224,2022-09-04 00:00:00+00:00,google,positive,673,"Place, Entities [Entity Service], Brand, Produ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13569,1536065922309443586,807095,Google placed an engineer on paid leave after ...,517,2376,721,54860244,862,2022-06-12 00:00:00+00:00,google,negative,-3614,"Entities [Entity Service], Brand, Product, Int..."
13570,1579700083314282496,200394155,"Well done, blockchain address is now available...",172,3689,588,2701,3338,2022-10-11 00:00:00+00:00,google,positive,4449,"Entities [Entity Service], Business Taxonomy, ..."
13571,1568970566690029568,1318329572660752385,IU is #1 The Most Searched Female Kpop Idol of...,16,701,206,758,87,2022-09-11 00:00:00+00:00,google,positive,923,"Entities [Entity Service], Business Taxonomy, ..."
13572,1602312836113375232,39440257,I made this on TikTok &amp; thought I’d share ...,4,381,58,15208,974,2022-12-12 00:00:00+00:00,google,positive,443,"Entities [Entity Service], Business Taxonomy, ..."


## Show descriptive statistics using describe()
Use the ```pandas DataFrame.describe()```
[method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html)
to generate descriptive statistics. Descriptive statistics include those that
summarize the central tendency, dispersion and shape of a dataset’s
distribution, excluding ```NaN``` values. You may also use other Python methods
to interact with your data.

In [184]:
results.describe()

,like_count,retweet_count,quote_count,followers_count,following_count,virality
count,13574.0,13574.0,13574.0,13574.0,13574.0,13574.0
mean,86.375055,4238.971342,692.550759,2261918.536393,3824.741933,1041.566893
std,417.487818,15686.291869,2369.77011,8381355.573403,21964.600149,14764.016028
min,0.0,55.0,2.0,0.0,0.0,-336692.0
25%,7.0,671.0,107.0,11437.5,262.0,0.0
50%,17.0,1217.0,208.0,65492.0,826.0,0.0
75%,48.0,2697.5,476.0,338632.75,1900.0,1389.75
max,19672.0,561233.0,85287.0,69721199.0,1071680.0,653049.0


In [185]:
from pandas._libs.lib import is_date_array
id_twitter=results["id"].values.tolist()
annotation_twitter = results["context"].apply(lambda x: x.strip().split())
date_list= results['tweet_date'].dt.strftime('%Y-%m-%d').tolist()

In [186]:
import pandas as pd
import numpy as np

# Create a sample dataframe with rows and a list of annotations
df = pd.DataFrame({'id': id_twitter, 'annotations': annotation_twitter , 'date' : date_list })

# Convert each list of annotations to a set
df['annotations'] = df['annotations'].apply(set)

# Create a column for the length of the set of annotations
df['annotations_len'] = df['annotations'].apply(len)

# Create a column for the set of annotations as a string
df['annotations_str'] = df['annotations'].apply(lambda x: ' '.join(x))

# # Create a sparse matrix of the annotations
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
# annotations_sparse = vectorizer.fit_transform(df['annotations_str'])

# # Compute the dot product of the sparse matrix with its transpose
# dot_product = annotations_sparse.dot(annotations_sparse.T)

# # Create an array of the lengths of the sets of annotations
# annotations_len = np.array(df['annotations_len'])

# # Compute the Jaccard similarity scores by dividing the dot product by the outer product of the lengths of the sets of annotations
# jaccard_similarities = dot_product / (annotations_len.reshape(-1, 1) + annotations_len - dot_product)

# # Convert the resulting numpy array to a dataframe
# jaccard_similarities_df = pd.DataFrame(jaccard_similarities, index=df['id'], columns=df['id'])

# # Print the Jaccard similarity scores
# print(jaccard_similarities)


In [187]:
df = df.drop_duplicates(subset='id')

# Group the dataframe by date
grouped_df = df.groupby('date')

# Initialize an empty list to store the Jaccard similarity scores for each date
average_scores_df = pd.DataFrame()

# Iterate over each group
for date, group in grouped_df:
    # Convert each list of annotations to a set
    group['annotations'] = group['annotations'].apply(set)

    # Create a column for the length of the set of annotations
    group['annotations_len'] = group['annotations'].apply(len)

    # Create a column for the set of annotations as a string
    group['annotations_str'] = group['annotations'].apply(lambda x: ' '.join(x))

    # Create a sparse matrix of the annotations
    annotations_sparse = vectorizer.fit_transform(group['annotations_str'])

    # Compute the dot product of the parsing matrix with its transpose
    dot_product = annotations_sparse.dot(annotations_sparse.T)

    # Create an array of the lengths of the sets of annotations
    annotations_len = np.array(group['annotations_len'])

    # Compute the Jaccard similarity scores by dividing the dot product by the outer product of the lengths of the sets of annotations
    jaccard_similarities = dot_product / (annotations_len.reshape(-1, 1) + annotations_len - dot_product)

    # Convert the resulting numpy array to a dataframe
    jaccard_similarities_df = pd.DataFrame(jaccard_similarities, index=group['id'], columns=group['id'])
    average_scores = jaccard_similarities_df.mean(axis=1)
    average_scores_d = pd.DataFrame({'id': average_scores.index, 'average_score': average_scores.values})
    average_scores_df = pd.concat([average_scores_df,average_scores_d], ignore_index=True)

# Print the average Jaccard similarity scores dataframe
print(average_scores_df)
result = average_scores_df


                        id  average_score
0      1477238765596913666       0.425420
1      1477200740254945280       0.682453
2      1477406027129790469       0.682453
3      1477251232486068228       0.682453
4      1477212109402304513       0.682453
...                    ...            ...
13569  1609176578021380100       0.618988
13570  1609053623576846337       0.618988
13571  1609087187974328320       0.682517
13572  1609059452212346881       0.712176
13573  1609197581300817922       0.787428

[13574 rows x 2 columns]


In [188]:
# import networkx as nx

# import matplotlib.colors as mcolors


# # Create an empty graph
# G = nx.Graph()

# # Add nodes to the graph for each row in the dataframe
# for i in range(len(df)):
#     G.add_node(df.loc[i, 'id'])

# # Add edges to the graph for each pair of rows with a Jaccard similarity above a certain threshold
# threshold = 0.3
# for i, row in average_scores_df.iterrows():
#     for j, sim in row.iteritems():
#         if i != j and sim > threshold:
#             G.add_edge(i, j, weight=sim)


# # Define a color map that maps the dates to a color range from light blue to purple
# cmap = mcolors.LinearSegmentedColormap.from_list('date_cmap', ['lightblue', 'purple'], N=365)

# # Create a list of node colors based on the date
# node_colors = [df.loc[df['id'] == n, 'date'].values[0] for n in G.nodes()]

# # Normalize the date to a value between 0 and 1
# min_date = pd.to_datetime(df['date']).min()
# max_date = pd.to_datetime(df['date']).max()
# normalized_dates = (pd.to_datetime(node_colors) - min_date) / (max_date - min_date)


# # Map the normalized date to a color
# node_colors = cmap(normalized_dates)

# # Draw the graph using a spring layout
# nx.draw_spring(G, with_labels=False,node_color=node_colors)

# # Show the graph
# import matplotlib.pyplot as plt
# plt.show()


In [189]:
result.to_csv('data.csv') 

In [1]:
# import pandas as pd

# # create an empty DataFrame to store the merged data
# merged_data = pd.DataFrame()

# # list of file names
# file_names = ['Jaccard_similarities_day_amazon.csv', 'Jaccard_similarities_day_apple.csv', 'Jaccard_similarities_day_cocacola.csv', 'Jaccard_similarities_day_disney.csv', 'Jaccard_similarities_day_google.csv', 'Jaccard_similarities_day_intel.csv', 'Jaccard_similarities_day_mcdonalds.csv', 'Jaccard_similarities_day_s&p500.csv', 'Jaccard_similarities_day_tesla.csv', 'Jaccard_similarities_day_twitter.csv', 'Jaccard_similarities_day_walmart.csv']

# # loop through the file names and read each file into a DataFrame
# for file in file_names:
#     data = pd.read_csv(file)
#     merged_data = pd.concat([merged_data, data], ignore_index=True)

# # save the merged data to a new CSV file
# merged_data.to_csv('Jacard_Data.csv', index=False)
